In [1]:
import os 
import pandas as pd
import sys
import glob

In [2]:
#import the gene table, which has the NLRs for all accessions 
gene_table='/global/home/users/chandlersutherland/e16/Maize_NLRome_GeneTable.txt'
pd.read_csv(gene_table, sep='\t')

,Gene,Clade_0,Clade_1,Clade_2,Clade_3,Clade,Ecotype,Allele,File,HV
0,ZM00029AB319340_P001,Int3448_22,Int3448_22,NaN,NaN,Int3448_22,ki3,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int3448_22.afa,0
1,ZM00023AB313710_P001,Int3448_22,Int3448_22,NaN,NaN,Int3448_22,cml247,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int3448_22.afa,0
2,ZM00042AB469660_P001,Int3448_22,Int3448_22,NaN,NaN,Int3448_22,tzi8,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int3448_22.afa,0
3,ZM00001EB304860_P001,Int3448_22,Int3448_22,NaN,NaN,Int3448_22,b73,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int3448_22.afa,0
4,ZM00022AB310170_P001,Int3448_22,Int3448_22,NaN,NaN,Int3448_22,cml228,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int3448_22.afa,0
...,...,...,...,...,...,...,...,...,...,...
3491,ZM00030AB305770_P001,Int6844_19,Int6844_19,NaN,NaN,Int6844_19,ki11,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int6844_19.afa,0
3492,ZM00032AB317850_P001,Int6844_19,Int6844_19,NaN,NaN,Int6844_19,m37w,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int6844_19.afa,0
3493,ZM00042AB318810_P001,Int6844_19,Int6844_19,NaN,NaN,Int6844_19,tzi8,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int6844_19.afa,0
3494,ZM00039AB310760_P001,Int6844_19,Int6844_19,NaN,NaN,Int6844_19,oh43,1,RAxML_tree_pbNB-ARC/Initial_Clades/Int6844_19.afa,0


In [7]:
#glob all the gff3 files across all genomes 
gff3_files=glob.glob('/global/scratch/users/chandlersutherland/e16/*/genome/*1.gff3')

#glob the fulldata.txt files 
fulldata=glob.glob('/global/scratch/users/chandlersutherland/e16/*/genome/*fulldata.txt')

In [8]:
len(gff3_files)

33

In [5]:
b73='/global/scratch/users/chandlersutherland/e16/B73/genome/Zm-B73-REFERENCE-NAM-5.0_Zm00001eb.1.gff3'
gff3=pd.read_csv(b73, sep='\t', skiprows=3, header=0, lineterminator='\n')
gff3=gff3.reset_index()
gff3=gff3.rename(columns={gff3.columns[0]:'chr', gff3.columns[1]:'source', gff3.columns[2]:'sequence_ontology', gff3.columns[3]:'start', gff3.columns[4]:'end', gff3.columns[5]:'score', gff3.columns[6]:'strand', gff3.columns[7]:'phase', gff3.columns[8]:'attributes'})
gff3

,chr,source,sequence_ontology,start,end,score,strand,phase,attributes
0,chr1,assembly,chromosome,1,308452471,.,.,.,ID=1;Name=chromosome:Zm-B73-REFERENCE-NAM-5.0:...
1,chr1,NAM,gene,34617,40204,.,+,.,ID=Zm00001eb000010;biotype=protein_coding;logi...
2,chr1,NAM,mRNA,34617,40204,.,+,.,ID=Zm00001eb000010_T001;Parent=Zm00001eb000010...
3,chr1,NAM,five_prime_UTR,34617,34721,.,+,.,Parent=Zm00001eb000010_T001
4,chr1,NAM,exon,34617,35318,.,+,.,Parent=Zm00001eb000010_T001;Name=Zm00001eb0000...
...,...,...,...,...,...,...,...,...,...
1143798,scaf_695,NAM,mRNA,2336,4226,.,-,.,ID=Zm00001eb443030_T001;Parent=Zm00001eb443030...
1143799,scaf_695,NAM,five_prime_UTR,4218,4226,.,-,.,Parent=Zm00001eb443030_T001
1143800,scaf_695,NAM,exon,2336,4226,.,-,.,Parent=Zm00001eb443030_T001;Name=Zm00001eb4430...
1143801,scaf_695,NAM,CDS,2898,4217,.,-,0,ID=Zm00001eb443030_P001;Parent=Zm00001eb443030...


In [5]:
genes=gff3[gff3['sequence_ontology']=='gene']
genes['attributes'].str.split(';', expand=True)[2].unique()

array(['logic_name=cshl_gene'], dtype=object)

In [14]:
#define a function that takes a gff3 file path as input, and outputs an all gene bed file and an NLR specific bed file. 
def bed_generator(gff_filepath, gene_table):
    #read in and cleanup gff3 file 
    gff3=pd.read_csv(gff_filepath, sep='\t', skiprows=3, header=0, lineterminator='\n')
    gff3=gff3.reset_index()
    gff3=gff3.rename(columns={gff3.columns[0]:'chr', gff3.columns[1]:'source', gff3.columns[2]:'sequence_ontology', gff3.columns[3]:'start', gff3.columns[4]:'end', gff3.columns[5]:'score', gff3.columns[6]:'strand', gff3.columns[7]:'phase', gff3.columns[8]:'attributes'})

    #get metadata
    accession=gff_filepath.split('/')[6]
    basename=gff_filepath.split('/')[8].split('.')[0]
    print(accession+ ' gff3 file loaded')
    
    #filter gff3 to just genes, and expand attributes  
    genes = gff3[gff3['sequence_ontology'] == 'gene']
    attributes=genes.iloc[:,8].str.split(';',expand=True)
    genes.loc[:,'ID']=attributes.iloc[:,0].str.split('=', expand=True)[1]
    genes.loc[:,'biotype']=attributes.iloc[:,1].str.split('=', expand=True)[1]
    genes.loc[:,'logic_name']=attributes.iloc[:,2].str.split('=', expand=True)[1]
    
    #generate a bed file for all genes 
    bed = genes.loc[:,['chr', 'start', 'end', 'ID', 'strand']]
    bed = bed.rename(columns={'chr':'chrom', 'start':'chromStart', 'end':'chromEnd', 'ID':'name', 'strand':'strand'})
    bed.to_csv('/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_all_gene.bed', sep='\t')
    print('all gene bed written to '+'/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_all_gene.bed')
    
    #Load NLR gene names
    gene=pd.read_csv(gene_table,sep = '\t')
    nlr_df=gene[gene['Ecotype'] == accession.lower()]
    nlrs=nlr_df.loc[:,'Gene'].str.split('_', expand=True).iloc[:,0].str.replace('ZM', 'Zm').str.replace('AB', 'ab').str.replace('EB','eb').unique()
    
    #quality stats 
    match=bed['name'].isin(nlrs).sum()
    unmatch=len(nlrs)-match 
    print(str(match)+' NLRs found in gff file for '+accession+'. We are missing '+str(unmatch)+'.')
    
    #write out NLR bed file
    nlr_bed=bed[bed['name'].isin(nlrs)]
    nlr_bed.to_csv('/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_NLR.bed', sep='\t')
    print('NLR bed written to '+'/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_NLR.bed')
    
    

In [10]:
for i in gff3_files:
    bed_generator(str(i), gene_table)

B97 gff3 file loaded


/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


all gene bed written to /global/scratch/users/chandlersutherland/e16/B97/genome/Zm-B97-REFERENCE-NAM-1_all_gene.bed
125 NLRs found in gff file for B97. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/B97/genome/Zm-B97-REFERENCE-NAM-1_NLR.bed
Oh43 gff3 file loaded
all gene bed written to /global/scratch/users/chandlersutherland/e16/Oh43/genome/Zm-Oh43-REFERENCE-NAM-1_all_gene.bed
131 NLRs found in gff file for Oh43. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/Oh43/genome/Zm-Oh43-REFERENCE-NAM-1_NLR.bed
Tx303 gff3 file loaded
all gene bed written to /global/scratch/users/chandlersutherland/e16/Tx303/genome/Zm-Tx303-REFERENCE-NAM-1_all_gene.bed
121 NLRs found in gff file for Tx303. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/Tx303/genome/Zm-Tx303-REFERENCE-NAM-1_NLR.bed
CML69 gff3 file loaded
all gene bed written to /global/scratch/users/chandlersutherland/e16/CML69/genome/Zm-CML6

In [15]:
bed_generator(gff3_files[10], gene_table)

B73 gff3 file loaded
all gene bed written to /global/scratch/users/chandlersutherland/e16/B73/genome/Zm-B73-REFERENCE-NAM-5_all_gene.bed
126 NLRs found in gff file for B73. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/B73/genome/Zm-B73-REFERENCE-NAM-5_NLR.bed


Take a look at the fulldata.txt files, which should have common gene names. 

In [29]:
test='/global/scratch/users/chandlersutherland/e16/B73/genome/Zm00001eb.1.fulldata.txt'
test_csv=pd.read_csv(test, sep='\t', header=0, lineterminator='\n', names=['gene_model', 'assembly_version', 'annotation_version', 'transcript_count', 'canonical_transcript_name', 'tandem_count', 'chr', 'gm_start', 'gm_end', 'locus_symbol', 'locus_name', 'gene_products', 'obo_terms', 'obo_names'])

In [61]:
with open(test, 'r') as file: 
    file_contents=file.readlines()
    for row in file_contents:
        if row.find('Zm00001d023265') != -1:
            print(row)
#test_csv['locus_symbol']=test_csv['locus_symbol'].str.strip('         ')

In [56]:
#test_csv.to_csv('/global/scratch/users/chandlersutherland/e16/B73/genome/gene_names.txt')
test_csv[test_csv['gene_model']=='Zm00001eb405930']

,gene_model,assembly_version,annotation_version,transcript_count,canonical_transcript_name,tandem_count,chr,gm_start,gm_end,locus_symbol,locus_name,gene_products,obo_terms,obo_names
40647,Zm00001eb405930,Zm-B73-REFERENCE-NAM-5.0,Zm00001eb.1,2,Zm00001eb405930_T002,3,chr10,3333939,3335714,rp1,resistance to Puccinia sorghi1 ...,"{""nucleotide-binding site (NBS)-leucine-rich r...","{GO:0043531,GO:0005524,GO:0006952,GO:0043531,G...","{""ADP binding"",""ATP binding"",""defense response..."


In [19]:
#define a function that takes a gff3 file path as input, and outputs hv and non-hv NLR bed files 
def bed_generator_hv(gff_filepath, gene_table):
    #read in and cleanup gff3 file 
    gff3=pd.read_csv(gff_filepath, sep='\t', skiprows=3, header=0, lineterminator='\n')
    gff3=gff3.reset_index()
    gff3=gff3.rename(columns={gff3.columns[0]:'chr', gff3.columns[1]:'source', gff3.columns[2]:'sequence_ontology', gff3.columns[3]:'start', gff3.columns[4]:'end', gff3.columns[5]:'score', gff3.columns[6]:'strand', gff3.columns[7]:'phase', gff3.columns[8]:'attributes'})

    #get metadata
    accession=gff_filepath.split('/')[6]
    basename=gff_filepath.split('/')[8].split('.')[0]
    print(accession+ ' gff3 file loaded')
    
    #filter gff3 to just genes, and expand attributes  
    genes = gff3[gff3['sequence_ontology'] == 'gene']
    attributes=genes.iloc[:,8].str.split(';',expand=True)
    genes.loc[:,'ID']=attributes.iloc[:,0].str.split('=', expand=True)[1]
    genes.loc[:,'biotype']=attributes.iloc[:,1].str.split('=', expand=True)[1]
    genes.loc[:,'logic_name']=attributes.iloc[:,2].str.split('=', expand=True)[1]
    
    bed = genes.loc[:,['chr', 'start', 'end', 'ID', 'strand']]
    bed = bed.rename(columns={'chr':'chrom', 'start':'chromStart', 'end':'chromEnd', 'ID':'name', 'strand':'strand'})
    
    #Load gene names
    gene=pd.read_csv(gene_table,sep = '\t')
    nlr_df=gene[gene['Ecotype'] == accession.lower()]
    nlrs=nlr_df.loc[:,'Gene'].str.split('_', expand=True).iloc[:,0].str.replace('ZM', 'Zm').str.replace('AB', 'ab').str.replace('EB','eb').unique()
    
    hv_nlrs=nlr_df[nlr_df['HV']==1].loc[:,'Gene'].str.split('_', expand=True).iloc[:,0].str.replace('ZM', 'Zm').str.replace('AB', 'ab').str.replace('EB','eb').unique()
    nhv_nlrs=nlr_df[nlr_df['HV']==0].loc[:,'Gene'].str.split('_', expand=True).iloc[:,0].str.replace('ZM', 'Zm').str.replace('AB', 'ab').str.replace('EB','eb').unique()
    #quality stats 
    match=bed['name'].isin(hv_nlrs).sum()
    unmatch=len(hv_nlrs)-match 
    print(str(match)+' hvNLRs found in gff file for '+accession+'. We are missing '+str(unmatch)+'.')
    match=bed['name'].isin(nhv_nlrs).sum()
    unmatch=len(nhv_nlrs)-match 
    print(str(match)+' non-hvNLRs found in gff file for '+accession+'. We are missing '+str(unmatch)+'.')   
    #write out NLR bed file
    hv_nlr_bed=bed[bed['name'].isin(hv_nlrs)]
    hv_nlr_bed.to_csv('/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_hvNLR.bed', sep='\t', index=False)
    print('NLR bed written to '+'/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_hvNLR.bed')
    
    nhv_nlr_bed=bed[bed['name'].isin(nhv_nlrs)]
    nhv_nlr_bed.to_csv('/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_nhvNLR.bed', sep='\t', index=False)
    print('NLR bed written to '+'/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_nhvNLR.bed')

In [20]:
for i in gff3_files:
    bed_generator_hv(str(i), gene_table)

B97 gff3 file loaded
12 hvNLRs found in gff file for B97. We are missing 0.
113 non-hvNLRs found in gff file for B97. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/B97/genome/Zm-B97-REFERENCE-NAM-1_hvNLR.bed
NLR bed written to /global/scratch/users/chandlersutherland/e16/B97/genome/Zm-B97-REFERENCE-NAM-1_nhvNLR.bed
Oh43 gff3 file loaded
11 hvNLRs found in gff file for Oh43. We are missing 0.
120 non-hvNLRs found in gff file for Oh43. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/Oh43/genome/Zm-Oh43-REFERENCE-NAM-1_hvNLR.bed
NLR bed written to /global/scratch/users/chandlersutherland/e16/Oh43/genome/Zm-Oh43-REFERENCE-NAM-1_nhvNLR.bed
Tx303 gff3 file loaded
11 hvNLRs found in gff file for Tx303. We are missing 0.
110 non-hvNLRs found in gff file for Tx303. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/Tx303/genome/Zm-Tx303-REFERENCE-NAM-1_hvNLR.bed
NLR bed written to /global/scratc

HP301 gff3 file loaded
15 hvNLRs found in gff file for HP301. We are missing 0.
101 non-hvNLRs found in gff file for HP301. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/HP301/genome/Zm-HP301-REFERENCE-NAM-1_hvNLR.bed
NLR bed written to /global/scratch/users/chandlersutherland/e16/HP301/genome/Zm-HP301-REFERENCE-NAM-1_nhvNLR.bed
CML277 gff3 file loaded
11 hvNLRs found in gff file for CML277. We are missing 0.
115 non-hvNLRs found in gff file for CML277. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/CML277/genome/Zm-CML277-REFERENCE-NAM-1_hvNLR.bed
NLR bed written to /global/scratch/users/chandlersutherland/e16/CML277/genome/Zm-CML277-REFERENCE-NAM-1_nhvNLR.bed
Ki3 gff3 file loaded
17 hvNLRs found in gff file for Ki3. We are missing 0.
112 non-hvNLRs found in gff file for Ki3. We are missing 0.
NLR bed written to /global/scratch/users/chandlersutherland/e16/Ki3/genome/Zm-Ki3-REFERENCE-NAM-1_hvNLR.bed
NLR bed written

Repeat but with exons to create a bed file for methylation processing 

In [9]:
#define a function that takes a gff3 file path as input, and outputs an all exon bed file
def bed_generator_exon(gff_filepath):
    #read in and cleanup gff3 file 
    gff3=pd.read_csv(gff_filepath, sep='\t', skiprows=3, header=0, lineterminator='\n')
    gff3=gff3.reset_index()
    gff3=gff3.rename(columns={gff3.columns[0]:'chr', gff3.columns[1]:'source', gff3.columns[2]:'sequence_ontology', gff3.columns[3]:'start', gff3.columns[4]:'end', gff3.columns[5]:'score', gff3.columns[6]:'strand', gff3.columns[7]:'phase', gff3.columns[8]:'attributes'})

    #get metadata
    accession=gff_filepath.split('/')[6]
    basename=gff_filepath.split('/')[8].split('.')[0]
    print(accession+ ' gff3 file loaded')

    exons = gff3[gff3['sequence_ontology'] == 'exon']

    attributes=exons.iloc[:,8].str.split(';',expand=True)
    exons.loc[:,'ID']=attributes.iloc[:,0].str.split('=', expand=True)[1]
    exons.loc[:,'biotype']=attributes.iloc[:,1].str.split('=', expand=True)[1]
    exons.loc[:,'logic_name']=attributes.iloc[:,2].str.split('=', expand=True)[1]

    bed = exons.loc[:,['chr', 'start', 'end', 'ID', 'biotype', 'strand']]
    bed = bed.rename(columns={'chr':'chrom', 'start':'chromStart', 'end':'chromEnd', 'ID':'name', 'biotype':'exon', 'strand':'strand'})

    bed.to_csv('/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_all_exon.bed', sep='\t')
    print('all exon bed written to '+'/global/scratch/users/chandlersutherland/e16/' + accession + '/genome/' + basename + '_all_exon.bed')

In [11]:
for i in gff3_files:
    bed_generator_exon(str(i))
    
exon_files=glob.glob('/global/scratch/users/chandlersutherland/e16/*/genome/*_all_exon.bed')
len(exon_files)

B97 gff3 file loaded


/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


all exon bed written to /global/scratch/users/chandlersutherland/e16/B97/genome/Zm-B97-REFERENCE-NAM-1_all_exon.bed
Oh43 gff3 file loaded
all exon bed written to /global/scratch/users/chandlersutherland/e16/Oh43/genome/Zm-Oh43-REFERENCE-NAM-1_all_exon.bed
Tx303 gff3 file loaded
all exon bed written to /global/scratch/users/chandlersutherland/e16/Tx303/genome/Zm-Tx303-REFERENCE-NAM-1_all_exon.bed
CML69 gff3 file loaded
all exon bed written to /global/scratch/users/chandlersutherland/e16/CML69/genome/Zm-CML69-REFERENCE-NAM-1_all_exon.bed
CML333 gff3 file loaded
all exon bed written to /global/scratch/users/chandlersutherland/e16/CML333/genome/Zm-CML333-REFERENCE-NAM-1_all_exon.bed
KI11 gff3 file loaded
all exon bed written to /global/scratch/users/chandlersutherland/e16/KI11/genome/Zm-Ki11-REFERENCE-NAM-1_all_exon.bed
NC358 gff3 file loaded
all exon bed written to /global/scratch/users/chandlersutherland/e16/NC358/genome/Zm-NC358-REFERENCE-NAM-1_all_exon.bed
TZI8 gff3 file loaded
all exo

33